In [21]:
import pandas as pd
import numpy as np
import random as rd
import re, math, functools
import nltk
import requests
from nltk import pos_tag, bigrams
from nltk.corpus import stopwords as stp
from bs4 import BeautifulSoup
from sklearn.metrics import confusion_matrix

In [14]:
def features_extraction(text, rule_1, rule_2, no_match):
    
    wc = []
    append = wc.append
    text = text.lower()
    tokens = re.split("\W+", text)
    stopwords = stp.words("english")
    remove = tokens.remove
    while "" in tokens:
        remove("")
    for token in tokens:
        if token in stopwords:
            remove(token)
    token_pos = pos_tag(tokens)
    
    for i in xrange(len(token_pos)-1):
        if (token_pos[i][1], token_pos[i+1][1]) not in rule_1 and\
            (token_pos[i][1], token_pos[i+1][1]) not in rule_2:
                continue
        elif (token_pos[i][1], token_pos[i+1][1]) in rule_1:
            append((token_pos[i][0], token_pos[i+1][0]))
        elif (token_pos[i][1], token_pos[i+1][1]) in rule_2:
            try:
                if token_pos[i+2] not in no_match:
                    append((token_pos[i][0], token_pos[i+1][0]))
            except IndexError:
                append((token_pos[i][0], token_pos[i+1][0]))
        else:
            continue

    return wc

In [15]:
def Semantic_Orientation(phrases,positive="excellent", negative="poor", prior = 1,engine="google",distance=10,threshold=None):
  
    so_positive = 0.01
    so_negative = 0.01
    so_avg = 0
    get = requests.get

    engine.lower() == "google"
    url = "http://www.google.com/search?q=%s"
    id_pattern = "resultStats"
    
    for phrase in phrases:
        term = "%22{}+{}%22+AROUND({})+%22{}%22".format(phrase[0], distance, phrase[1], positive)
        page = get(url % term)
        soup = BeautifulSoup(page.text, "lxml")
        try:
            rtr_pos = int("".join(re.split("\D+",soup.find("div", id=id_pattern).get_text().encode("utf-８"))))
        except AttributeError:
            rtr_pos = 0
            
        # if rtr_pos == None:
        #     rtr_pos = 0
        
        term = "%22{}+{}%22+AROUND({})+%22{}%22".format(phrase[0], distance, phrase[1], negative)
        page = get(url % term)
        soup = BeautifulSoup(page.text, "lxml")
        try:
            rtr_neg = int("".join(re.split("\D+",soup.find("div", id=id_pattern).get_text().encode("utf-８"))))
        except AttributeError:
            rtr_neg = 0
            
        # if rtr_neg == None:
         #   rtr_neg =0
            
        so_positive += rtr_pos
        so_negative += rtr_neg
        
        so_avg += math.log(so_positive/so_negative/prior)
    
    #if len(phrases)==0:
     #   so_avg=0
    #else:
        so_avg = so_avg*1.0/len(phrases)
    
    if threshold == None:
        return so_avg
    else:
        return int(so_avg>threshold)

In [40]:
yelp_data = pd.read_csv('yelp.csv')

# Spliting into high and low
yelp_data["Rating_final ( 1 for High, 0 For Low)"] = np.where((yelp_data['stars'].isin([4,5])),1.0,0.0)

# Take only a random subset of 2000 rows
yelp_subset = yelp_data.sample(frac=.1,random_state=91)
yelp_subset = yelp_subset.reset_index()
yelp_subset = yelp_subset.drop('index',axis=1)
yelp_subset[:3]

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Indian,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others,Review,"Rating_final ( 1 for High, 0 For Low)"
0,2,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,I have been to Windsor twice now once for hap...,0.0
1,3,0,1,3,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,Food is good. Don t take kids though.Kids past...,0.0
2,1,2,1,2,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,I usually poke fun at Olive Garden as having b...,0.0


In [41]:
# Defining Patters for Feature Extraction
rule_1 = [("JJ", "NN"), ("JJ", "NNS"), ("RB", "VB"), ("RB", "VBD"), ("RB", "VBN"), ("RB", "VBG"),("RBR", "VB"), ("RBR", "VBD"), ("RBR", "VBN"), ("RBR", "VBG"),
          ("RBS", "VB"), ("RBS", "VBD"), ("RBS", "VBN"), ("RBS", "VBG")]
rule_2 = [("RB", "JJ"), ("RBR", "JJ"), ("RBS", "JJ"),("JJ", "JJ"),("NN", "JJ"), ("NNS", "JJ")]
no_match = ["NN", "NNS"]

# Extracting Features

features_extract = functools.partial(features_extraction, rule_1=rule_1, rule_2=rule_2, no_match=no_match)
features=yelp_subset.Review.map(features_extract)

In [42]:
# Calling the function that gives Semantic Orientation for a review.

Opinion_Orientation = functools.partial(Semantic_Orientation,positive="excellent", negative="poor",prior = 1,engine="google",distance=5,threshold=None)

# Mapping the function to our review list ( Sample : 12)

prediction = features[:12].map(Opinion_Orientation)

In [43]:
# Checking accuracy
y_pred = prediction.tolist()
y_true = yelp_subset['Rating_final ( 1 for High, 0 For Low)'][:12].tolist()

cm_sk = confusion_matrix(y_true, y_pred)
cm = nltk.ConfusionMatrix(y_true, y_pred)


print (cm.pretty_format(sort_by_count=True, show_percents=True))
if cm_sk[1][1] ==None:
    sum_sk[1][1]=0;
print 'The accuracy is:',round(float(cm_sk[1][1]+cm_sk[0][0])/float(len(y_pred)),3)



    |      1      0 |
    |      .      . |
    |      0      0 |
----+---------------+
1.0 |     <.> 58.3% |
0.0 |      . <41.7%>|
----+---------------+
(row = reference; col = test)

The accuracy is: 0.417


In [44]:
y_true

[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]